# 05 — YOLOv8 Object Detection (Bird vs Drone)
This notebook covers:

- Install & import YOLOv8 (ultralytics)
- Verify dataset structure & annotations (YOLO format)
- Create `config/data.yaml`
- Train a YOLOv8 model (choose yolov8n/yolov8s)
- Validate & save best weights
- Run inference on test images & custom images
- Save detection outputs to `reports/figures/yolo_predictions/`

## Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setup Paths

In [ ]:
import os

# Base project directory (customized as provided by user)
BASE_DIR = "/content/drive/MyDrive/Project-2/Aerial_Object_Classification_Detection"

# Location of YOLO object detection dataset
OD_DIR = os.path.join(BASE_DIR, "data", "object_detection_Dataset")

# Directory for YOLO config files (including data.yaml)
CONFIG_DIR = os.path.join(BASE_DIR, "config")

# Directory where trained YOLO models will be stored
MODELS_DIR = os.path.join(BASE_DIR, "models")

# Directory to save visual inference results (bounding box images)
REPORTS_YOLO_DIR = os.path.join(BASE_DIR, "reports", "figures", "yolo_predictions")

# Create folders if they do not exist
os.makedirs(CONFIG_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(REPORTS_YOLO_DIR, exist_ok=True)

print("Base Directory:", BASE_DIR)
print("Object Detection Dataset:", OD_DIR)
print("Config Directory:", CONFIG_DIR)
print("Models Directory:", MODELS_DIR)
print("YOLO Predictions Directory:", REPORTS_YOLO_DIR)

Base Directory: /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection
Object Detection Dataset: /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/data/object_detection_Dataset
Config Directory: /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/config
Models Directory: /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/models
YOLO Predictions Directory: /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/reports/figures/yolo_predictions


## Install YOLOv8

In [3]:
# This installs the official ultralytics package, which includes
# YOLOv8 and tools for detection, classification, segmentation.

!pip install -U ultralytics --quiet

# Print YOLO version for confirmation
from ultralytics import __version__ as yolo_version
print("YOLOv8 Version:", yolo_version)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
YOLOv8 Version: 8.3.230


In [4]:
import os
import glob

# Base object detection dataset dir (set earlier in Cell 1)
OD_DIR = os.path.join(BASE_DIR, "data", "object_detection_Dataset")
print("Inspecting object detection dataset at:", OD_DIR)

# Define expected Layout B paths (canonical variables used by later cells)
YOLO_IMAGES_TRAIN = os.path.join(OD_DIR, "train", "images")
YOLO_LABELS_TRAIN = os.path.join(OD_DIR, "train", "labels")
YOLO_IMAGES_VAL   = os.path.join(OD_DIR, "valid",   "images")
YOLO_LABELS_VAL   = os.path.join(OD_DIR, "valid",   "labels")
YOLO_IMAGES_TEST  = os.path.join(OD_DIR, "test",  "images")
YOLO_LABELS_TEST  = os.path.join(OD_DIR, "test",  "labels")

Inspecting object detection dataset at: /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/data/object_detection_Dataset


## Imports

In [5]:
import os
import glob
import yaml
from PIL import Image
import matplotlib.pyplot as plt
from ultralytics import YOLO

## Verify YOLO Dataset Structure

In [6]:

# YOLO expects this structure (your structure):
# object_detection_Dataset/
#   train/images
#   train/labels
#   val/images
#   val/labels
#   test/images
#   test/labels

import os
import glob

# Base dir already defined: OD_DIR = <your dataset folder>

# Define reversed (correct) structure paths
images_train = os.path.join(OD_DIR, "train", "images")
labels_train = os.path.join(OD_DIR, "train", "labels")
images_val   = os.path.join(OD_DIR, "valid",   "images")
labels_val   = os.path.join(OD_DIR, "valid",   "labels")
images_test  = os.path.join(OD_DIR, "test",  "images")
labels_test  = os.path.join(OD_DIR, "test",  "labels")



# Helper to preview folder contents
def list_preview(path, limit=8):
    files = sorted(glob.glob(os.path.join(path, "*")))[:limit]
    print(f"\n📁 {path} (showing {len(files)} items):")
    for f in files:
        print(" -", f)

# Preview folders
list_preview(images_train)
list_preview(labels_train)
list_preview(images_val)
list_preview(labels_val)
list_preview(images_test)
list_preview(labels_test)

# Find a sample annotation file
sample_label_file = None
for p in [labels_train, labels_val, labels_test]:
    txts = glob.glob(os.path.join(p, "*.txt"))
    if txts:
        sample_label_file = txts[0]
        break

# Show example annotation
if sample_label_file:
    print("\n📄 Sample Annotation File:", sample_label_file)
    print("First lines:")
    with open(sample_label_file, "r") as f:
        for ln in f.read().splitlines()[:5]:
            print(" >", ln)
    print("\nAnnotation format: <class_id> <x_center> <y_center> <width> <height>  (normalized)")
else:
    print("⚠️ No label files found. Ensure YOLO label TXT files exist.")



📁 /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/data/object_detection_Dataset/train/images (showing 8 items):
 - /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/data/object_detection_Dataset/train/images/000596bc75926abe_jpg.rf.49cf79ededcab42c160f599b1104f421.jpg
 - /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/data/object_detection_Dataset/train/images/000596bc75926abe_jpg.rf.dd331440f1cc9338d6c4ff82cd7200d6.jpg
 - /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/data/object_detection_Dataset/train/images/000712bbe0a4844f_jpg.rf.39ceec229b870fda18f3d8fb1f09af87.jpg
 - /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/data/object_detection_Dataset/train/images/000712bbe0a4844f_jpg.rf.8e869da274a69f828a31be97b34d8a32.jpg
 - /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/data/object_detection_Dataset/train/images/000f6e054dde03c8_

###  Create `config/data.yaml`
Create a compact YOLO `data.yaml` pointing to train/val/test folders, class count and names. This file is used by YOLO for training/validation/inference.

In [7]:
import yaml
import os

# path where YAML will be saved
data_yaml_path = os.path.join(CONFIG_DIR, "data.yaml")

# Compose YAML content using Layout B (train/images etc.)
data_yaml = {
    "path": OD_DIR,                  # base path to dataset (Layout B root)
    "train": "train/images",         # relative to `path`
    "val":   "valid/images",        # Changed 'valid' to 'val'
    "test":  "test/images",
    "nc": 2,                         # number of classes
    "names": ["bird", "drone"]       # class names
}

# Write YAML to disk
os.makedirs(CONFIG_DIR, exist_ok=True)
with open(data_yaml_path, "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print("Wrote data.yaml ->", data_yaml_path)
print("\nContents preview:")
with open(data_yaml_path, "r") as f:
    print(f.read())

Wrote data.yaml -> /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/config/data.yaml

Contents preview:
names:
- bird
- drone
nc: 2
path: /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/data/object_detection_Dataset
test: test/images
train: train/images
val: valid/images



## Choose YOLO Model Variant & Hyperparameters

Pick a YOLO variant & basic hyperparameters. Defaults below are safe for Colab. Adjust if you have more GPU RAM.

In [8]:
# Set model variant and training hyperparameters in one place.
# Edit these values if you need a different tradeoff between speed and accuracy.

# Model choices: "yolov8n" (nano), "yolov8s" (small), "yolov8m" (medium), etc.
model_name = "yolov8n"   # default: nano (fast). Change to "yolov8s" for better accuracy.

# Training image size (pixels), epochs, batch size (reduce if OOM)
imgsz = 640
epochs = 30
batch = 8

# Where YOLO will save runs (project folder)
YOLO_RUN_DIR = os.path.join(MODELS_DIR, "yolo_runs")
os.makedirs(YOLO_RUN_DIR, exist_ok=True)

# Print chosen configuration
print("YOLO config:")
print(" model_name:", model_name)
print(" imgsz:", imgsz)
print(" epochs:", epochs)
print(" batch:", batch)
print(" YOLO runs dir:", YOLO_RUN_DIR)

YOLO config:
 model_name: yolov8n
 imgsz: 640
 epochs: 30
 batch: 8
 YOLO runs dir: /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/models/yolo_runs


## Train YOLOv8 Model

Train YOLOv8 using the Python API. Training artifacts (including `best.pt`) are saved under `models/yolo_runs/<run_name>/`.

In [9]:
# ------------------------------------------------------------
# Cell 7 — Train YOLOv8 model (CPU/GPU Auto Selection)
# ------------------------------------------------------------
# Automatically uses GPU if available. If no GPU (Colab T4 not enabled),
# training will safely fall back to CPU with reduced settings.

import torch
from ultralytics import YOLO

# 1) Auto-detect device
device_setting = 0 if torch.cuda.is_available() else "cpu"
print("🔍 Device selected for training:", device_setting)

# 2) Auto-adjust hyperparameters if CPU (to avoid extremely slow training)
if device_setting == "cpu":
    print("⚠️ CPU detected — reducing batch size and image size for faster training.")
    imgsz = 416     # lower resolution for CPU
    batch = 4       # very small batch for CPU
else:
    print("✅ GPU available — using original settings.")

# 3) Load YOLO backbone
yolo_model = YOLO(f"{model_name}.pt")

# 4) Training run name
run_name = f"{model_name}_bird_drone"

# 5) Start training
print("\n🚀 Starting YOLOv8 training...")
results = yolo_model.train(
    data=data_yaml_path,  # from Cell 5
    imgsz=imgsz,
    epochs=epochs,
    batch=batch,
    project=YOLO_RUN_DIR,
    name=run_name,
    exist_ok=True,
    device=device_setting,   # "cpu" or GPU index
    workers=4
)

print("\n🎉 Training complete! Check run folder for logs & weights:")
print(os.path.join(YOLO_RUN_DIR, run_name))


🔍 Device selected for training: cpu
⚠️ CPU detected — reducing batch size and image size for faster training.

🚀 Starting YOLOv8 training...
Ultralytics 8.3.230 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/config/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model

## Locate & Copy Best Weights

Find `best.pt` produced by YOLO and copy it into `models/` with a deterministic name so later steps can load it.

In [10]:
# After training, YOLO saves run outputs under project/<run_name>/
# This cell finds the best weights and copies them to a stable path under MODELS_DIR.

import glob
import shutil

# Run directory we used for training
run_dir = os.path.join(YOLO_RUN_DIR, run_name)
print("Searching for best.pt under:", run_dir)

# Common YOLO locations for best.pt
candidates = glob.glob(os.path.join(run_dir, "**", "weights", "best.pt"), recursive=True)
if not candidates:
    # fallback: search any best.pt under run_dir
    candidates = glob.glob(os.path.join(run_dir, "**", "best.pt"), recursive=True)

if candidates:
    best_src = candidates[0]
    best_dst = os.path.join(MODELS_DIR, f"yolov8_{model_name}_bird_drone_best.pt")
    shutil.copy(best_src, best_dst)
    print("Copied best weights to:", best_dst)
else:
    print("Could not find best.pt automatically. Check the YOLO run directory manually:", run_dir)
    best_dst = None

Searching for best.pt under: /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/models/yolo_runs/yolov8n_bird_drone
Copied best weights to: /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/models/yolov8_yolov8n_bird_drone_best.pt


## Validation (mAP, Precision, Recall)

Run YOLO validation to compute mAP, precision, recall on the dataset defined in `data.yaml`. Results are printed by ultralytics.

In [12]:
# Uses best weights copied earlier (best_dst). If no GPU is available,
# it will run on CPU and reduce batch/imgsz if necessary for speed.

import torch
from ultralytics import YOLO
import os

# Ensure best weights variable exists and file is present
if 'best_dst' not in locals() or best_dst is None or not os.path.exists(best_dst):
    raise FileNotFoundError("best weights not found. Run Cell 8 or set best_dst to the correct .pt file path.")

# Auto-select device: GPU index 0 if available, otherwise "cpu"
device_setting = 0 if torch.cuda.is_available() else "cpu"
print("Selected device for validation:", device_setting)
print("torch.cuda.is_available():", torch.cuda.is_available(), "torch.cuda.device_count():", torch.cuda.device_count())

# If running on CPU, you may want to reduce batch/imgsz to avoid long runtimes
if device_setting == "cpu":
    print("⚠️ Running on CPU — adjusting batch/imgsz for practicality.")
    # make local copies so we don't overwrite training settings (optional)
    val_batch = min(4, batch if 'batch' in globals() else 4)
    val_imgsz = min(416, imgsz if 'imgsz' in globals() else 416)
else:
    val_batch = batch if 'batch' in globals() else 16
    val_imgsz = imgsz if 'imgsz' in globals() else 640

print(f"Validation params -> imgsz: {val_imgsz}, batch: {val_batch}")

# Load model and run validation
print("Validating model:", best_dst)
val_model = YOLO(best_dst)

# Run validation (this prints metrics: mAP@0.5, mAP@0.5:0.95, precision, recall)
results = val_model.val(data=data_yaml_path, imgsz=val_imgsz, batch=val_batch, device=device_setting)

print("Validation finished. Key metrics printed above by ultralytics.")

Selected device for validation: cpu
torch.cuda.is_available(): False torch.cuda.device_count(): 0
⚠️ Running on CPU — adjusting batch/imgsz for practicality.
Validation params -> imgsz: 416, batch: 4
Validating model: /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/models/yolov8_yolov8n_bird_drone_best.pt
Ultralytics 8.3.230 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 1.7±1.2 ms, read: 4.6±3.1 MB/s, size: 25.6 KB)
val: Scanning /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/data/object_detection_Dataset/valid/labels.cache... 448 images, 6 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 448/448 433.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 112/112 1.7it/s 1:06
                   all        448        663      0.876      0.726      0

## Run Inference on Test Images & Save Visualizations

Run detection on test images and save annotated images (bounding boxes) to `reports/figures/yolo_predictions/`.

In [13]:
# Load the detection model (best weights) and run inference on test images.
# Save annotated images (with boxes) to REPORTS_YOLO_DIR.

from ultralytics import YOLO
from PIL import Image
import glob
import os

# Ensure best weights exist
if not ( 'best_dst' in locals() and best_dst and os.path.exists(best_dst) ):
    raise FileNotFoundError("Best weights not found (best_dst). Run Cell 8 and ensure file exists.")

det_model = YOLO(best_dst)

# Collect test images under Layout B test/images
test_images_dir = os.path.join(OD_DIR, "test", "images")
test_images = sorted(glob.glob(os.path.join(test_images_dir, "**", "*.jpg"), recursive=True) +
                     glob.glob(os.path.join(test_images_dir, "**", "*.png"), recursive=True))

print(f"Found {len(test_images)} test images. Processing up to first 200 for speed.")

os.makedirs(REPORTS_YOLO_DIR, exist_ok=True)

for i, img_path in enumerate(test_images[:200]):
    # Run prediction (returns a list of Results objects)
    res = det_model.predict(source=img_path, imgsz=imgsz, conf=0.25, save=False)
    # Get annotated image as numpy array
    annotated = res[0].plot()
    out_path = os.path.join(REPORTS_YOLO_DIR, f"pred_{i}_{os.path.basename(img_path)}")
    Image.fromarray(annotated).save(out_path)
    if (i + 1) % 25 == 0:
        print(f"Saved {i+1} annotated images...")

print("Saved inference outputs to:", REPORTS_YOLO_DIR)

Found 224 test images. Processing up to first 200 for speed.

image 1/1 /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/data/object_detection_Dataset/test/images/00083b384685315d_jpg.rf.abfd1b2cc8c681777bae66d5327bb9ea.jpg: 416x416 1 bird, 216.5ms
Speed: 3.8ms preprocess, 216.5ms inference, 19.7ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/data/object_detection_Dataset/test/images/00188d7f40a84793_jpg.rf.7f9da2b662dc236fbdcc1f22d8e0983e.jpg: 416x416 1 bird, 183.1ms
Speed: 2.1ms preprocess, 183.1ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /content/drive/MyDrive/Project-2/Aerial_Object_Classification_&_Detection/data/object_detection_Dataset/test/images/0028adf0e92c3da2_jpg.rf.48c97d30547a46e83abcbd1bf801c72f.jpg: 416x416 1 bird, 213.4ms
Speed: 2.0ms preprocess, 213.4ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)

imag

## Inference on Developer Uploaded Example

Run one detection on the developer-provided image and save + display the annotated output.

In [14]:
# Uses the developer-provided image local path (we will also print it as the "url").
# The path will be used by downstream tooling if needed.

uploaded_image = "/mnt/data/ee1ddb95-23dd-4388-b0d8-3cb2b408bf0e.png"  # developer-provided local path
print("Uploaded example path (local):", uploaded_image)

# Ensure model loaded
if not ( 'best_dst' in locals() and best_dst and os.path.exists(best_dst) ):
    raise FileNotFoundError("Best weights not found (best_dst). Run Cell 8 first.")

det_model = YOLO(best_dst)

if not os.path.exists(uploaded_image):
    print("⚠️ Uploaded example not found at path:", uploaded_image)
else:
    # Run prediction on the uploaded example
    res = det_model.predict(source=uploaded_image, imgsz=imgsz, conf=0.25, save=False)
    annotated = res[0].plot()
    # Display inline (Colab) and save to reports folder
    from IPython.display import display
    display(Image.fromarray(annotated))

    out_custom = os.path.join(REPORTS_YOLO_DIR, "pred_uploaded_example.png")
    Image.fromarray(annotated).save(out_custom)
    print("Saved annotated uploaded example to:", out_custom)

    # Print the uploaded image path again as the "url" to be used by tools later
    print("\nUse this file path as the uploaded-file URL for downstream tooling:")
    print("uploaded_file_url:", uploaded_image)

Uploaded example path (local): /mnt/data/ee1ddb95-23dd-4388-b0d8-3cb2b408bf0e.png
⚠️ Uploaded example not found at path: /mnt/data/ee1ddb95-23dd-4388-b0d8-3cb2b408bf0e.png
